# 0. Getting the model

If we don't already have the repository with the model, we can clone it as instructed by their authors in https://github.com/sanghyun-son/EDSR-PyTorch

First we will define a directory to clone our repository. We will use our current working directory as our "base" directory, and we will clone the repository in a folder "EDSR-PyTorch" inside.

In [1]:
import os

dir_base = os.getcwd()
dir_edsrpytorch = os.path.join(dir_base,"EDSR-PyTorch")     # ./EDSR-PyTorch

Now we can clone the repository from GitHub:

In [2]:
# Cloning the repository as instructed in https://github.com/sanghyun-son/EDSR-PyTorch
!git clone https://github.com/thstkdgus35/EDSR-PyTorch {dir_edsrpytorch}

Cloning into '/content/EDSR-PyTorch'...
remote: Enumerating objects: 806, done.
remote: Total 806 (delta 0), reused 0 (delta 0), pack-reused 806
Receiving objects: 100% (806/806), 63.09 MiB | 11.88 MiB/s, done.
Resolving deltas: 100% (516/516), done.


It will also be useful to define the path to the src folder inside of the repository with the model, to use later on.

In [3]:
dir_src = os.path.join(dir_edsrpytorch,"src")               # ./EDSR-PyTorch/src

We also need a pretrained model, in case we don't have one already:

In [7]:
import os
import urllib.request

# Pre-train folder
dir_pretrain = os.path.join(dir_edsrpytorch, "pre-train")      # ./EDSR-PyTorch/pre-train
if not os.path.exists(dir_pretrain):
    os.makedirs(dir_pretrain, exist_ok=True)

# Pretrained model
pretrain_model = "edsr_x4-4f62e9ef.pt"
pretrain_model_path = os.path.join(dir_pretrain, pretrain_model)

# Download it if not present
if not os.path.isfile(pretrain_model_path):
  url = "https://cv.snu.ac.kr/research/EDSR/models/edsr_x4-4f62e9ef.pt"
  with urllib.request.urlopen(url) as response, open(pretrain_model_path, 'wb') as out_file:
    data = response.read()
    out_file.write(data)
  print(f"Pretrained model {pretrain_model} has been downloaded inside {dir_pretrain}")
else :
  print(f"Using pretrained model {pretrain_model_path}")


Pretrained model edsr_x4-4f62e9ef.pt has been downloaded inside /content/EDSR-PyTorch/pre-train


# 1. Visualizing the model by printing it

Below we will show a simply demonstration on how we could see the model structure by printing it with print(model).

We will change the current working directory to the src folder of the model.

In [8]:
## Change the current working directory to EDSR-PyTorch/src, to run the model
import os

os.chdir(dir_src)
print(f"The Current working directory is: {os.getcwd()}")

The Current working directory is: /content/EDSR-PyTorch/src


We can change the current working directory back to the base directory at any time by uncommenting and running:

In [7]:
#os.chdir(dir_base)
#print(f"The Current working directory is: {os.getcwd()}")

The Current working directory is: /content


## 1.1 Setting up the args

The model requires arguments to initializate. Given we are not using the model as expected on a terminal in order to see its structure, we must also set the parameters before hand with the code below.

NOTE: If using in Google Colab, the option.py in the src folder requires an additional line of code regarding the parsing of the arguments. If this line has not been previously added, it can be add by running the code below:

In [6]:
"""
NOTE: This will always try to read the option.py file from a file
named as option-backup.py, which corresponds to the original
option.py from the EDSR-PyTorch repository.

If such file is not found, the code below will assume it's a fresh
clone of the repository, and it will make it.
"""

import os

# Line we want to add
new_line = 'parser.add_argument("-f", "--file", required=False)\n\n'

# File paths
option_file_path = os.path.join(dir_src,'option.py')                 # Original
option_file_path_backup = os.path.join(dir_src,'option-backup.py')   # Back-up

# If the back-up file is there, read it, or make it if not
if os.path.isfile(option_file_path_backup):
  # Read the content of the back-up file
  with open(option_file_path_backup, 'r') as file:
      lines = file.readlines()
else:
  # Read the content of the original file
  with open(option_file_path, 'r') as file:
      lines = file.readlines()
  # Write the content to the back-up file
  with open(option_file_path_backup, 'w') as file:
      file.writelines(lines)

# Insert the new line at the desired location (between lines 145 and 146)
lines.insert(145, new_line)

# Write the modified content back to the file
with open(option_file_path, 'w') as file:
    file.writelines(lines)

print(f"Option file successfully updated in {option_file_path}")

Option file successfully updated in /content/EDSR-PyTorch/src/option.py


We can set up a custom templated EDSR_custom by modifying the corresponding lines below:

In [7]:
"""
NOTE: This will always try to read the template.py file from a file
named as template-backup.py, which corresponds to the original
template.py from the EDSR-PyTorch repository.

If such file is not found, the code below will assume it's a fresh
clone of the repository, and it will make it.
"""

# Lines to add to the file
template = [
    '    if args.template.find(\'EDSR_custom\') >= 0:\n',
    '        args.dir_data =  \"../image-data\"\n',
    '        args.data_train = \"Custom\"\n',
    '        args.data_test = \"Custom\"\n',
    '        args.data_range = \"1-2400/2401-2500\"\n',
    '        args.ext = \"sep\"\n',
    '        args.scale = \"4\"\n',
    '        args.model = \"EDSR\"\n',
    '        args.pre_train = \"../pre-train/edsr_x4-4f62e9ef.pt\"\n',
    '        args.n_resblocks = 32\n',
    '        args.n_feats = 256\n',
    '        args.res_scale = 0.1\n',
    '        args.test_every = 100\n',
    '        args.epochs = 11\n',
    '        args.batch_size = 16\n',
    '        args.save = \"edsr_x4-train\"\n'
]
#Note: spaces/tabulations are important.

# Specify the file path
template_file_path = os.path.join(dir_src,'template.py')

# Have a back-up of the original template.py
template_file_path_backup = os.path.join(dir_src,'template-backup.py')

# If the back-up file is there, read it, or make it if not
if os.path.isfile(template_file_path_backup):
  # Read the content of the back-up file
  with open(template_file_path_backup, 'r') as file:
      lines = file.readlines()
else:
  # Read the content of the original file
  with open(template_file_path, 'r') as file:
      lines = file.readlines()
  # Write the content to the back-up file
  with open(template_file_path_backup, 'w') as file:
      file.writelines(lines)

# Insert the new lines at the desired location (on line 54)
for idx,line in enumerate(template):
  lines.insert(54+idx, line)

# Write the modified content back to the file
with open(template_file_path, 'w') as file:
    file.writelines(lines)

print(f"Template successfully added to {template_file_path}")

Template successfully added to /content/EDSR-PyTorch/src/template.py


And we can load it by doing:

In [8]:
from option import args
import template

# Use the desired template of args:
args.template = "EDSR_custom"
template.set_template(args)

# Set-up args as in the option.py file, after setting the template:
args.scale = list(map(lambda x: int(x), args.scale.split('+')))
args.data_train = args.data_train.split('+')
args.data_test = args.data_test.split('+')

if args.epochs == 0:
  args.epochs = 1e8

for arg in vars(args):
  if vars(args)[arg] == 'True':
    vars(args)[arg] = True
  elif vars(args)[arg] == 'False':
    vars(args)[arg] = False

# Set-up additional parameters
args.chop = True
args.save_results = True
args.param_to_freeze = ""
args.body_to_freeze = ""
args.tail_to_freeze = ""
args.print_frozen_param = True
args.save_models_each = 3

## 1.2 Loading and printing the model

We can load and print the model with the code below.

In [9]:
import torch

import utility
import model

torch.manual_seed(args.seed)
checkpoint = utility.checkpoint(args)

# Load the model
model = model.Model(args, checkpoint)

# Print it
print(model)

Making model...
Load the model from ../pre-train/edsr_x4-4f62e9ef.pt
Model(
  (model): EDSR(
    (sub_mean): MeanShift(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (add_mean): MeanShift(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (head): Sequential(
      (0): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (body): Sequential(
      (0): ResBlock(
        (body): Sequential(
          (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (1): ResBlock(
        (body): Sequential(
          (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (2): ResBlock(
        (body): Sequential(
          (0): Conv2d(256, 256, kernel_size=(3, 3), stride=

# Visualizing the model with torchinfo

NOTE: If torchinfo is not installed, we can install it in Google Colab by doing:

In [10]:
# Checking for torchinfo
try:
  from torchinfo import summary
except:
  !pip install -q torchinfo
  from torchinfo import summary

We can use the function summary to see the structure of the loaded model.

In [12]:
from torchinfo import summary

batch_size = 16
image_dim = (3, 64, 64)

summary(model=model,
        input_size=(batch_size, *image_dim),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        cache_forward_pass=True,
        depth=5,
        mode="train",
        row_settings=["var_names"],
        idx_scale=0
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
Model (Model)                            [16, 3, 64, 64]      [16, 3, 256, 256]    --                   Partial
├─EDSR (model)                           [16, 3, 64, 64]      [16, 3, 256, 256]    --                   Partial
│    └─MeanShift (sub_mean)              [16, 3, 64, 64]      [16, 3, 64, 64]      (12)                 False
│    └─Sequential (head)                 [16, 3, 64, 64]      [16, 256, 64, 64]    --                   True
│    │    └─Conv2d (0)                   [16, 3, 64, 64]      [16, 256, 64, 64]    7,168                True
│    └─Sequential (body)                 [16, 256, 64, 64]    [16, 256, 64, 64]    --                   True
│    │    └─ResBlock (0)                 [16, 256, 64, 64]    [16, 256, 64, 64]    --                   True
│    │    │    └─Sequential (body)       [16, 256, 64, 64]    [16, 256, 64, 64]    590,080              True
│    │ 

# 3. Quick test

We could make a quick application of the model if we have a pre-trained model (state dictionary file in .pt format) and some JPG/PNG images.

In [5]:
# Parameters
import os

# Images must be inside this folder
dir_demo = os.path.join(dir_edsrpytorch, "test")     # Test image provided in the EDSR-PyTorch repository

# Pretrained model
pretrain_path = os.path.join(dir_edsrpytorch, "pre-train", "edsr_x4-4f62e9ef.pt")   #Pretrained EDSR x4 by the authors

data_test = "Demo"
scale = 4
save = "test"

In [9]:
# Command to try the model
!python main.py --data_test {data_test} --dir_demo {dir_demo} --scale {scale} --save {save} --n_resblocks 32 --n_feats 256 --res_scale 0.1 --pre_train {pretrain_path} --test_only --save_results

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Making model...
Load the model from /content/EDSR-PyTorch/pre-train/edsr_x4-4f62e9ef.pt

Evaluation:
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
  0%|                                                     | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will cr

If we used the 0853x4.png image inside of ./EDSR-PyTorch/test, we can see its SR 4-times upscaling 0853x4_x4_SR.png inside of ./EDSR.PyTorch/experiment/test/results-Demo, as well as the config.txt and log.txt inside of ./EDSR.PyTorch/experiment/test

# 4. Training

In order to train the model, we need:


* The EDSR-PyTorch repository
* A corresponding pre-trained model, if we want to start (or resume) the training from one
* The dataset to use (properly structured and placed in disk)
* [Optional] To already have been made the modifications to the original code, if we want to use the new arguments and modules produced in our work
* To prepare a custom template in the template.py OR provide all the pertinent argument to the script while running the command



In particular, for the EDSR x4, we should provide the arguments:

* dir_data = Path to the folder with the dataset
* data_train = Name of the module for training
* data_test = Name of the module for testing
* data_range = Range of images to train/test
* ext = "sep" (to use binaries of images) or "img" (to use images directly)
* scale = 4
* model = EDSR
* pre_train = Path to the pretrained model to use (or empty if we are training from scratch)
* n_resblocks = 32
* n_feats = 256
* res_scale = 0.1
* epochs = Number of epochs + 1 (note: to train for 10 epochs, for example, this argument should be 11)
* batch_size = For example, 16
* save = Name to give to the folder to store the results of the training in the "experiment" folder of the repository

We could manually set a template inside the template.py file, or do it as shown above in section 1.1.



With all set, we can train the model by running the following command on a terminal:

In [1]:
# Command to train the model
!python main.py --template EDSR_custom --save_models --chop

python3: can't open file '/content/main.py': [Errno 2] No such file or directory
